In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os

In [2]:
## working directory 설정
new_working_directory = "/Users/hyunbin/Library/CloudStorage/OneDrive-Personal/05 빅데이터 전문가 과정/01 통계 데이터 사이언스/00 팀플/final/new"
os.chdir(new_working_directory)

print("Current Working Directory:", os.getcwd())

Current Working Directory: /Users/hyunbin/Library/CloudStorage/OneDrive-Personal/05 빅데이터 전문가 과정/01 통계 데이터 사이언스/00 팀플/final/new


# 가. 데이터 정제

## 01. 데이터 불러오기

In [3]:
## raw data
train_raw = pd.read_csv("./00 data/lending_club_2020_train.csv")
test_raw = pd.read_csv("./00 data/lending_club_2020_test.csv")

train_data = train_raw.copy()
test_data = test_raw.copy()

/var/folders/yb/vl5rylzx0dv4gxwytw3c2h080000gn/T/ipykernel_4667/3669893079.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  train_raw = pd.read_csv("./00 data/lending_club_2020_train.csv")


In [4]:
## 대출 직전 데이터
trd_feat = pd.read_csv("./00 data/trd_features.csv")
trd_col = trd_feat['feature'].tolist()
trd_col.append("issue_d")
trd_col

['funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_length',
 'annual_inc',
 'loan_status',
 'dti',
 'delinq_2yrs',
 'fico_avg',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'last_fico_range_high',
 'last_fico_range_low',
 'acc_now_delinq',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'inq_fi',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'mort_acc',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'home_ownership_MORTGAGE',
 'home_ownership_NON

## 02. 1차 정제 : train data

In [5]:
## loan_status : 부도 여부, 타겟 변수
# non-default = 0, default = 1
# 'Fully Paid'와 'Charged Off'가 아닌 행 제거
train_data = train_data[train_data['loan_status'].isin(['Fully Paid', 'Charged Off'])]
train_data['loan_status'] = np.where(train_data['loan_status'] == 'Fully Paid', 0, 1)

## fico_avg (파생변수)
# data cleansing. add avg fico score column 
insert_loc = train_data.columns.get_loc('fico_range_low')
train_data.insert(insert_loc, 'fico_avg', (train_data['fico_range_low'] + train_data['fico_range_high']) / 2)

## emp_length
# 2 way of emp_length mapping.
# emp_length이 NaN인 값(무직)을 -1로 매핑, 나머지를 오름차순으로 0~10
# emp_length이 NaN인 값(무직)을 -1로 매핑, 나머지를 오름차순으로 1~11. 무직과 고용상태를 구분하기 위함
label_mapping = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10
}

train_data['emp_length'] = train_data['emp_length'].map(label_mapping)
train_data['emp_length'] = train_data['emp_length'].fillna(-1)

## term
# categorical variable Labeling
label_mapping = {
    ' 36 months': 36,
    ' 60 months': 60
}

train_data['term'] = train_data['term'].map(label_mapping)

## grade, sub_grade labeling
grade = np.array(train_data["grade"])
sub_grade = np.array(train_data["sub_grade"])

encoder = LabelEncoder()
grade_encoded = encoder.fit_transform(grade)
sub_grade_encoded = encoder.fit_transform(sub_grade)

train_data["grade"] = grade_encoded
train_data["sub_grade"] = sub_grade_encoded

## int_rate, revol_util
train_data["int_rate"] = train_data["int_rate"].str.rstrip("%").astype(float)
train_data["revol_util"] = train_data["revol_util"].str.rstrip("%").astype(float)

## Qualitative var labeling(nominal var. without ordinal var)
train_data = pd.get_dummies(train_data, columns=["home_ownership", "verification_status"], drop_first=True)

In [6]:
## 열추출
train_data = train_data[trd_col]
train_data

,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,loan_status,...,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d
0,6000.0,6000.0,36,7.97,187.94,0,4,2.0,45000.0,0,...,15000.0,56511.0,True,False,False,False,False,False,True,Dec-2017
2,23200.0,23200.0,60,24.99,680.82,4,23,10.0,110000.0,1,...,20300.0,291465.0,True,False,False,False,False,False,True,Jul-2016
5,16000.0,16000.0,36,7.07,494.55,0,1,-1.0,65000.0,1,...,59100.0,49339.0,True,False,False,False,False,False,False,Oct-2017
6,4500.0,4500.0,36,10.42,146.10,1,7,5.0,50000.0,0,...,9300.0,11970.0,True,False,False,False,False,False,False,May-2017
7,20000.0,20000.0,36,9.99,645.25,1,7,10.0,60000.0,0,...,11000.0,73090.0,True,False,False,False,False,True,False,Sep-2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755288,26500.0,26450.0,60,17.27,662.45,2,14,2.0,60000.0,0,...,18900.0,16784.0,True,False,False,False,False,False,True,Feb-2013
1755290,3600.0,3600.0,36,19.99,133.78,4,20,7.0,33800.0,1,...,6500.0,13375.0,True,False,False,False,False,False,False,Mar-2014
1755291,19000.0,18875.0,36,6.68,583.89,0,2,4.0,71000.0,0,...,17600.0,43550.0,True,False,False,False,False,False,False,Apr-2015
1755292,9600.0,9600.0,36,14.33,329.65,2,10,9.0,44000.0,0,...,4200.0,10414.0,True,False,False,False,False,False,True,May-2013


In [7]:
## 결측치 계산
# 각 컬럼의 결측치 개수 계산
missing_counts = train_data.isnull().sum()

# 결측치가 있는 컬럼만 선택
missing_counts = missing_counts[missing_counts > 0]

# 결측치 비율 계산 (전체 행 수에 대한 비율)
missing_ratios = missing_counts / len(train_data)

# 결측치 개수와 비율을 하나의 DataFrame으로 생성한 후, 내림차순 정렬
missing_train_data = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratios
}).sort_values(by='Missing Count', ascending=False)

missing_train_data

,Missing Count,Missing Ratio
inq_fi,507831,0.455091
inq_last_12m,507831,0.455091
num_tl_120dpd_2m,81165,0.072736
bc_util,41086,0.036819
percent_bc_gt_75,40779,0.036544
bc_open_to_buy,40505,0.036298
pct_tl_nvr_dlq,40504,0.036298
avg_cur_bal,40433,0.036234
num_rev_accts,40405,0.036209
num_il_tl,40404,0.036208


## 03. 1차 정제 : test data

In [8]:
## loan_status : 부도 여부, 타겟 변수
# non-default = 0, default = 1
# 'Fully Paid'와 'Charged Off'가 아닌 행 제거
test_data = test_data[test_data['loan_status'].isin(['Fully Paid', 'Charged Off'])]
test_data['loan_status'] = np.where(test_data['loan_status'] == 'Fully Paid', 0, 1)

## fico_range_avg
# data cleansing. add avg fico score column 
insert_loc = test_data.columns.get_loc('fico_range_low')
test_data.insert(insert_loc, 'fico_avg', (test_data['fico_range_low'] + test_data['fico_range_high']) / 2)

## emp_length
# 2 way of emp_length mapping.
# emp_length이 NaN인 값(무직)을 -1로 매핑, 나머지를 오름차순으로 0~10
# emp_length이 NaN인 값(무직)을 -1로 매핑, 나머지를 오름차순으로 1~11. 무직과 고용상태를 구분하기 위함
label_mapping = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10
}

test_data['emp_length'] = test_data['emp_length'].map(label_mapping)
test_data['emp_length'] = test_data['emp_length'].fillna(-1)

## term
# categorical variable Labeling
label_mapping = {
    ' 36 months': 36,
    ' 60 months': 60
}

test_data['term'] = test_data['term'].map(label_mapping)

## grade, sub_grade labeling
grade = np.array(test_data["grade"])
sub_grade = np.array(test_data["sub_grade"])

encoder = LabelEncoder()
grade_encoded = encoder.fit_transform(grade)
sub_grade_encoded = encoder.fit_transform(sub_grade)

test_data["grade"] = grade_encoded
test_data["sub_grade"] = sub_grade_encoded

## int_rate, revol_util
test_data["int_rate"] = test_data["int_rate"].str.rstrip("%").astype(float)
test_data["revol_util"] = test_data["revol_util"].str.rstrip("%").astype(float)

## Qualitative var labeling(nominal var. without ordinal var)
test_data = pd.get_dummies(test_data, columns=["home_ownership", "verification_status"], drop_first=True)


In [9]:
## 열추출
test_data = test_data[trd_col]
test_data

,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,loan_status,...,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d
0,20000.0,20000.0,36,6.03,608.72,0,0,6.0,125000.0,0,...,49000.0,46184.0,True,False,False,False,False,True,False,Mar-2013
3,12000.0,12000.0,60,14.64,283.22,2,12,0.0,60000.0,1,...,11400.0,5439.0,False,False,False,False,True,True,False,Mar-2014
6,15000.0,15000.0,36,5.32,451.73,0,0,1.0,75000.0,0,...,17400.0,18876.0,True,False,False,False,False,False,False,Feb-2016
7,20000.0,20000.0,36,13.05,674.37,1,9,1.0,87500.0,0,...,25900.0,56573.0,False,False,False,True,False,False,True,Aug-2013
10,11150.0,11150.0,36,13.99,381.03,2,13,3.0,63498.0,0,...,27500.0,140853.0,False,False,False,False,True,True,False,Apr-2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170192,6000.0,6000.0,36,6.99,185.24,0,1,4.0,62000.0,0,...,0.0,122354.0,False,False,False,False,True,True,False,Apr-2017
1170193,6000.0,6000.0,36,12.79,201.56,2,10,-1.0,34000.0,0,...,9100.0,40364.0,True,False,False,False,False,False,True,Sep-2016
1170194,9600.0,9600.0,36,9.49,307.48,1,6,10.0,60000.0,0,...,10000.0,2028.0,False,False,False,True,False,False,True,Jan-2015
1170195,8400.0,8400.0,36,24.85,333.32,4,22,0.0,30000.0,1,...,8000.0,0.0,False,False,False,False,True,False,False,Oct-2017


In [10]:
## 결측치 계산
# 각 컬럼의 결측치 개수 계산
missing_counts = test_data.isnull().sum()

# 결측치가 있는 컬럼만 선택
missing_counts = missing_counts[missing_counts > 0]

# 결측치 비율 계산 (전체 행 수에 대한 비율)
missing_ratios = missing_counts / len(test_data)

# 결측치 개수와 비율을 하나의 DataFrame으로 생성한 후, 내림차순 정렬
missing_test_data = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratios
}).sort_values(by='Missing Count', ascending=False)

missing_test_data

,Missing Count,Missing Ratio
inq_last_12m,338593,0.454827
inq_fi,338592,0.454826
num_tl_120dpd_2m,54452,0.073145
bc_util,27669,0.037167
percent_bc_gt_75,27401,0.036807
bc_open_to_buy,27235,0.036584
pct_tl_nvr_dlq,27177,0.036506
avg_cur_bal,27139,0.036455
num_tl_30dpd,27123,0.036434
num_rev_accts,27123,0.036434


## 04. 결측치 처리  

In [11]:
## missing data frame 만들기
# 열 이름 변경 (Rename Columns)
missing_train_data = missing_train_data.rename(columns={
    'Missing Count': 'missing_count_train',
    'Missing Ratio': 'missing_ratio_train'
})

missing_test_data = missing_test_data.rename(columns={
    'Missing Count': 'missing_count_test',
    'Missing Ratio': 'missing_ratio_test'
})

missing_df = pd.merge(missing_train_data, missing_test_data, left_index=True, right_index=True, how='outer')
missing_df

,missing_count_train,missing_ratio_train,missing_count_test,missing_ratio_test
acc_open_past_24mths,28301.0,0.025362,18980,0.025496
avg_cur_bal,40433.0,0.036234,27139,0.036455
bc_open_to_buy,40505.0,0.036298,27235,0.036584
bc_util,41086.0,0.036819,27669,0.037167
chargeoff_within_12_mths,38.0,0.000034,18,0.000024
dti,645.0,0.000578,463,0.000622
inq_fi,507831.0,0.455091,338592,0.454826
inq_last_12m,507831.0,0.455091,338593,0.454827
inq_last_6mths,NaN,NaN,1,0.000001
mort_acc,28301.0,0.025362,18980,0.025496


In [12]:
## missing_ratio >= 0.4 인 행 추출
train_filtered = missing_df[missing_df['missing_ratio_train'] >= 0.4]
test_filtered = missing_df[missing_df['missing_ratio_test'] >= 0.4]

## 공통되지 않은 행 추출 (Symmetric Difference)
diff_index = train_filtered.index.symmetric_difference(test_filtered.index)
different_rows = missing_df.loc[diff_index]
different_rows  # 비어있음

,missing_count_train,missing_ratio_train,missing_count_test,missing_ratio_test


In [13]:
## 40% 이상 결측치 있는 열 지우기
train_data = train_data.loc[:, (train_data.isnull().mean() < 0.4) | (train_data.columns == "loan_status")]
test_data = test_data.loc[:, (test_data.isnull().mean() < 0.4) | (test_data.columns == "loan_status")]

In [14]:
## 결측치 있는 행 제거
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

/var/folders/yb/vl5rylzx0dv4gxwytw3c2h080000gn/T/ipykernel_4667/2667964021.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.dropna(inplace=True)
/var/folders/yb/vl5rylzx0dv4gxwytw3c2h080000gn/T/ipykernel_4667/2667964021.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.dropna(inplace=True)


## 05. 데이터 내보내기

In [15]:
train_data.to_csv("./00 data/train_trd_timeline.csv", index=False)
test_data.to_csv("./00 data/test_trd_timeline.csv", index=False)

# 나. t-bill

In [16]:
## 데이터 불러오기
tbill = pd.read_csv('./00 data/t-bill_3M.csv')

In [17]:
## NaN 값이 있는 행 제거 후 datetime으로 변환
tbill = tbill.dropna(subset=['observation_date', 'DTB3'])  # observation_date와 DTB3 열의 NaN 값이 있는 행 제거
tbill['observation_date'] = pd.to_datetime(tbill['observation_date'])

/var/folders/yb/vl5rylzx0dv4gxwytw3c2h080000gn/T/ipykernel_4667/2612555112.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tbill['observation_date'] = pd.to_datetime(tbill['observation_date'])


In [18]:
## 각 달의 15일 데이터 또는 해당 달의 중앙값 사용
# 먼저 15일 데이터 추출
tbill_15 = tbill[tbill['observation_date'].dt.day == 15]

# 각 연도-월별 그룹 생성 및 처리
result_rows = []

# 각 그룹별로 처리
for (year, month), group in tbill.groupby([
    tbill['observation_date'].dt.year,
    tbill['observation_date'].dt.month
]):
    # 해당 연도-월에 15일 데이터가 있는지 확인
    has_15th = any((group['observation_date'].dt.day == 15))
    
    if has_15th:
        # 15일 데이터가 있으면 그대로 사용
        result_rows.append(group[group['observation_date'].dt.day == 15].iloc[0])
    else:
        # 15일 데이터가 없으면 해당 달의 데이터를 정렬하고 중앙값 선택
        sorted_group = group.sort_values('observation_date')
        middle_idx = len(sorted_group) // 2
        result_rows.append(sorted_group.iloc[middle_idx])

# 결과를 데이터프레임으로 변환하고 날짜순 정렬
tbill_15 = pd.DataFrame(result_rows)
tbill_15 = tbill_15.sort_values('observation_date').reset_index(drop=True)

In [19]:
## 날짜 형식 변경 (YYYY-MM-DD -> MMM-YYYY)
tbill_15['observation_date'] = tbill_15['observation_date'].dt.strftime('%b-%Y')

In [20]:
## CSV 파일로 저장
tbill_15.to_csv('./00 data/tbill_15_mod.csv', index=False)